In [65]:
import pandas as pd
from sqlalchemy import create_engine, Table, MetaData, insert, text
from collections import defaultdict
from datetime import date

In [66]:
DB_USER = 'postgres'
DB_PASSWORD = 'postgres123'
DB_HOST = '10.255.78.58'
DB_PORT = '9002'
DB_NAME = 'anomaly_detection'

connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

In [67]:
try:
    df = pd.read_sql(query, engine)
    print("✅ Query successful. Retrieved rows:", len(df))
    print(df.head())

except Exception as e:
    print("❌ Failed to query materialized view:", e)

✅ Query successful. Retrieved rows: 213213
  Transaction Date Sub-Merchant ID    sum
0         20250101     00944741669  32000
1         20250101      S240131329  74500
2         20250101     00987276177  40000
3         20250101     00920589109  20720
4         20250101     00913319126  28000


In [76]:
aggregation_cache = defaultdict(float)
def get_aggregation(sub_merchant_id, transaction_date):
    
    # query = f"""
    #     select "Transaction Date", "Sub-Merchant ID", sum("Transaction Amount") AS "Total Amount"
    #     from "BizPlay_2025_JAN" WHERE "Sub-Merchant ID" = '{sub_merchant_id}' and "Transaction Date"='{transaction_date}'
    #     GROUP BY "Sub-Merchant ID", "Transaction Date" ORDER BY "Transaction Date";
    #     """
    
    query = f"""
        select *
        from daily_thresholds_with_index
        WHERE "Sub-Merchant ID" = '{sub_merchant_id}' and "Transaction Date"='{transaction_date}';
    """
    df = pd.read_sql(query, engine)
    
    if not df.empty:
        # Extract values
        date_val = df.at[0, "Transaction Date"]
        sid = df.at[0, "Sub-Merchant ID"]
        amount = df.at[0, "Total Amount"]
        
        aggregation_cache[(date_val, sid)] = amount

In [69]:
def insert_into_database(data):
    try:
        trans_date = data["Transaction Date"]
        sid = data["Sub-Merchant ID"]
        amount = data["Transaction Amount"]
        key = (trans_date, sid)
    
        if key not in aggregation_cache:
            get_aggregation(sid, trans_date)
        
        insert_query = text("""
            INSERT INTO "BizPlay_2025_JAN" (
                "Transaction Date", "Transaction Time", "Transaction Amount", 
                "Customer Date of Birth", "Customer Gender", "Customer Age", 
                "Customer Region (City/County/District)", "Merchant Region (City/County/District)", 
                "Merchant ID", "Sub-Merchant ID", "Transaction Processing Agency (VAN)", "Distance"
            ) VALUES (
                :transaction_date, :transaction_time, :transaction_amount,
                :customer_dob, :customer_gender, :customer_age,
                :customer_region, :merchant_region,
                :merchant_id, :sub_merchant_id, :van, :distance
            );
        """)

        params = {
            "transaction_date": data["Transaction Date"],
            "transaction_time": data["Transaction Time"],
            "transaction_amount": data["Transaction Amount"],
            "customer_dob": data["Customer Date of Birth"],
            "customer_gender": data["Customer Gender"],
            "customer_age": data["Customer Age"],
            "customer_region": data["Customer Region (City/County/District)"],
            "merchant_region": data["Merchant Region (City/County/District)"],
            "merchant_id": data["Merchant ID"],
            "sub_merchant_id": data["Sub-Merchant ID"],
            "van": data["Transaction Processing Agency (VAN)"],
            "distance": data["Distance"]
        }
        with engine.begin() as conn:
            conn.execute(insert_query, params)
            # conn.commit()
        aggregation_cache[key] += data["Transaction Amount"]
        print("Inserted into Database.")
    except Exception as e:
        raise e

In [79]:
get_aggregation('00990981771', '20250201')
aggregation_cache

defaultdict(float, {('20250201', '00990981771'): 12200.0})

In [78]:
new_trans = {
    "Transaction Date": "20250201",
    "Transaction Time": "214209",
    "Transaction Amount": 12200,
    "Customer Date of Birth": "90522", 
    "Customer Gender":3, 
    "Customer Age": 15, 
    "Customer Region (City/County/District)":"41463",
    "Merchant Region (City/County/District)":"41463", 
    "Merchant ID":"F000000018", 
    "Sub-Merchant ID": "00990981771",
    "Transaction Processing Agency (VAN)":55, 
    "Distance":1
}
insert_into_database(new_trans)

Inserted into Database.
